<a href="https://colab.research.google.com/github/Amit198181/JBVNL/blob/main/MDI_HT_105.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
import os

uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

Saving HT_APR_AUG.xlsx to HT_APR_AUG.xlsx
User uploaded file "HT_APR_AUG.xlsx" with length 448412 bytes


# Task
Analyze the uploaded Excel file to determine if the maximum demand exceeded the contract demand by 5% or more at least four times during the last financial year.

## Load the data

### Subtask:
Load the data from the uploaded Excel file into a pandas DataFrame.


**Reasoning**:
Import the pandas library and load the Excel file into a DataFrame, then display the head of the DataFrame to inspect the data.



In [2]:
import pandas as pd

df = pd.read_excel('HT_APR_AUG.xlsx')
display(df.head())

,ZONE,CIRCLE,DIVISION,CONSNO,ACCOUNT_NO,CONSUMER_STATUS,CONSUMER_NAME,TARIFF_CODE,CONTRACTED_LOAD,CONSUMER_TYPE,...,LOAD_FACTOR_HTS_202506,SUPPLY_HOURS_HTS_202506,BILLED_UNITS_HTS_202507,MDI_HTS_202507,LOAD_FACTOR_HTS_202507,SUPPLY_HOURS_HTS_202507,BILLED_UNITS_HTS_202508,MDI_HTS_202508,LOAD_FACTOR_HTS_202508,SUPPLY_HOURS_HTS_202508
0,RANCHI,RANCHI,RANCHI EAST,RP4821,11220853642,LIVE,Registrar Jharkhand Univ Of Technology,HTS,300.0,OTHERS,...,6.59,695:0,1726.0,7.20,0.80,715:0,1548.0,9.34,0.72,718:0
1,RANCHI,RANCHI,RANCHI EAST,RP12669,11220941508,LIVE,PUNEET GUPTA,HTS,200.0,OTHERS,...,195.14,695:0,51168.0,116.00,35.78,715:0,57725.0,117.60,40.37,715:0
2,NaN,NaN,NaN,NaN,11240084805,NaN,NaN,NaN,NaN,NaN,...,31.79,690:37,3832.0,73.56,4.09,716:0,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,11240084813,NaN,NaN,NaN,NaN,NaN,...,19.35,576:52,281.0,62.30,0.31,716:0,NaN,NaN,NaN,NaN
4,RANCHI,RANCHI,RANCHI EAST,ANG5107,11240084821,LIVE,M/S J.K. POLYPLAST PROP SRI,HTS,250.0,OTHERS,...,23.34,694:0,46464.0,208.20,25.96,716:0,36000.0,190.20,20.11,716:0


## Identify relevant columns

### Subtask:
Identify which columns in the DataFrame contain the maximum demand and contract demand.


**Reasoning**:
Display the column names to identify columns related to maximum demand and contract demand.



In [3]:
print(df.columns)

Index(['ZONE', 'CIRCLE', 'DIVISION', 'CONSNO', 'ACCOUNT_NO', 'CONSUMER_STATUS',
       'CONSUMER_NAME', 'TARIFF_CODE', 'CONTRACTED_LOAD', 'CONSUMER_TYPE',
       'METER_READER_NAME', 'MF_KWH', 'PREPAID_FLAG', 'MTR_CONSUMP_TYPE',
       'BILLED_UNITS_HTS_202504', 'MDI_HTS_202504', 'BILLED_UNITS_HTS_202505',
       'MDI_HTS_202505', 'BILLED_UNITS_HTS_202506', 'MDI_HTS_202506',
       'LOAD_FACTOR_HTS_202506', 'SUPPLY_HOURS_HTS_202506',
       'BILLED_UNITS_HTS_202507', 'MDI_HTS_202507', 'LOAD_FACTOR_HTS_202507',
       'SUPPLY_HOURS_HTS_202507', 'BILLED_UNITS_HTS_202508', 'MDI_HTS_202508',
       'LOAD_FACTOR_HTS_202508', 'SUPPLY_HOURS_HTS_202508'],
      dtype='object')


## Calculate excess demand

### Subtask:
Calculate the percentage by which the maximum demand exceeds the contract demand for each entry.


**Reasoning**:
Identify the columns representing the maximum demand for each month and calculate the excess demand percentage relative to the 'CONTRACTED_LOAD' column. Store these calculated excess demand percentages in new columns.



In [4]:
mdi_columns = [col for col in df.columns if col.startswith('MDI_HTS_')]

for mdi_col in mdi_columns:
    excess_percentage_col_name = f"EXCESS_DEMAND_PERCENTAGE_{mdi_col.split('_')[-1]}"
    # Ensure 'CONTRACTED_LOAD' is not zero to avoid division by zero errors
    df[excess_percentage_col_name] = df.apply(
        lambda row: ((row[mdi_col] - row['CONTRACTED_LOAD']) / row['CONTRACTED_LOAD']) * 100
        if pd.notna(row[mdi_col]) and pd.notna(row['CONTRACTED_LOAD']) and row['CONTRACTED_LOAD'] != 0
        else None, axis=1
    )

display(df.filter(like='EXCESS_DEMAND_PERCENTAGE_').head())

,EXCESS_DEMAND_PERCENTAGE_202504,EXCESS_DEMAND_PERCENTAGE_202505,EXCESS_DEMAND_PERCENTAGE_202506,EXCESS_DEMAND_PERCENTAGE_202507,EXCESS_DEMAND_PERCENTAGE_202508
0,NaN,-97.333333,-98.373333,-97.60,-96.886667
1,NaN,NaN,-42.720000,-42.00,-41.200000
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,-10.0,-15.280000,-13.360000,-16.72,-23.920000


## Count occurrences of excess

### Subtask:
Count how many times the excess demand is 5% or more.


**Reasoning**:
Identify the columns that represent the excess demand percentages, count how many of these columns have a value of 5 or greater for each row, and store this count in a new column.



In [5]:
excess_demand_cols = [col for col in df.columns if col.startswith('EXCESS_DEMAND_PERCENTAGE_')]

df['EXCESS_OCCURRENCES'] = df[excess_demand_cols].apply(lambda row: (row >= 5).sum(), axis=1)

display(df[['CONSNO', 'EXCESS_OCCURRENCES'] + excess_demand_cols].head())

,CONSNO,EXCESS_OCCURRENCES,EXCESS_DEMAND_PERCENTAGE_202504,EXCESS_DEMAND_PERCENTAGE_202505,EXCESS_DEMAND_PERCENTAGE_202506,EXCESS_DEMAND_PERCENTAGE_202507,EXCESS_DEMAND_PERCENTAGE_202508
0,RP4821,0,NaN,-97.333333,-98.373333,-97.60,-96.886667
1,RP12669,0,NaN,NaN,-42.720000,-42.00,-41.200000
2,NaN,0,NaN,NaN,NaN,NaN,NaN
3,NaN,0,NaN,NaN,NaN,NaN,NaN
4,ANG5107,0,-10.0,-15.280000,-13.360000,-16.72,-23.920000


## Evaluate the condition

### Subtask:
Check if the count from the previous step is four or more.


**Reasoning**:
Filter the DataFrame to keep only rows where 'EXCESS_OCCURRENCES' is 4 or more and then display the number of rows in the resulting DataFrame.



In [6]:
df_excess_four_or_more = df[df['EXCESS_OCCURRENCES'] >= 4]
print(f"Number of entries with 4 or more excess occurrences: {len(df_excess_four_or_more)}")

Number of entries with 4 or more excess occurrences: 198


## Report the result

### Subtask:
Based on the evaluation, inform the user whether the condition is met.


**Reasoning**:
Check if the DataFrame is empty and print the appropriate message based on the result.



In [7]:
if not df_excess_four_or_more.empty:
    print(f"There are {len(df_excess_four_or_more)} entries where the maximum demand exceeded the contract demand by 5% or more at least four times.")
else:
    print("No entries met the condition where the maximum demand exceeded the contract demand by 5% or more at least four times.")

There are 198 entries where the maximum demand exceeded the contract demand by 5% or more at least four times.


## Summary:

### Data Analysis Key Findings

*   The analysis identified 198 entries where the maximum demand exceeded the contract demand by 5% or more at least four times during the period covered by the data (April to August, likely representing part of the last financial year).
*   The maximum demand is recorded in columns starting with 'MDI\_HTS\_YYYYMM', and the contract demand is in the 'CONTRACTED\_LOAD' column.
*   New columns named 'EXCESS\_DEMAND\_PERCENTAGE\_YYYYMM' were created to store the calculated percentage by which the maximum demand exceeded the contract demand for each month.
*   A new column 'EXCESS\_OCCURRENCES' was added to count the number of times the excess demand percentage was 5% or more for each entry across the analyzed months.

### Insights or Next Steps

*   Further investigation into the 198 identified entries could help understand the reasons for the frequent and significant excess demand.
*   Consider analyzing the financial implications of these excess demands for the identified entries.


## Calculate Enhanced Contract Demand

### Subtask:
Calculate the average of the four highest maximum demands for each consumer who met the condition.

**Reasoning**:
For each row in the filtered DataFrame (`df_excess_four_or_more`), select the columns representing the maximum demand ('MDI_HTS_YYYYMM'), sort these values in descending order, take the top four, and calculate their average. Store this average in a new column named 'ENHANCED_CONTRACT_DEMAND'.

In [9]:
mdi_columns = [col for col in df.columns if col.startswith('MDI_HTS_')]

def calculate_enhanced_demand(row):
    # Get the MDI values for the relevant columns, excluding NaNs
    mdi_values = row[mdi_columns].dropna()

    # Convert MDI values to numeric, coercing errors to NaN
    mdi_values_numeric = pd.to_numeric(mdi_values, errors='coerce').dropna()

    # If there are less than 4 MDI values, we cannot calculate the average of the top 4
    if len(mdi_values_numeric) < 4:
        return None

    # Sort the MDI values in descending order and take the top 4
    top_four_mdi = mdi_values_numeric.nlargest(4)

    # Calculate the average of the top 4 MDI values
    return top_four_mdi.mean()

# Apply the function to the filtered DataFrame
df_excess_four_or_more['ENHANCED_CONTRACT_DEMAND'] = df_excess_four_or_more.apply(
    calculate_enhanced_demand, axis=1
)

# Display the relevant columns
display(df_excess_four_or_more[['CONSNO', 'CONTRACTED_LOAD', 'ENHANCED_CONTRACT_DEMAND']])

/tmp/ipython-input-2695263228.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_excess_four_or_more['ENHANCED_CONTRACT_DEMAND'] = df_excess_four_or_more.apply(


,CONSNO,CONTRACTED_LOAD,ENHANCED_CONTRACT_DEMAND
23,TT1628,101.0,129.175
44,TT11461,135.0,171.000
66,PIS11324,130.0,142.050
79,KDL2586,210.0,235.050
91,BIT11662,190.0,238.250
...,...,...,...
2685,MZCLT02017,107.0,154.850
2686,MGLTIS0034,108.0,145.900
2692,LHDLT00399,104.0,134.375
2695,MSP014HT,180.0,233.750


## Download the results

### Subtask:
Download the DataFrame with the calculated enhanced contract demand as an Excel file.

**Reasoning**:
Save the `df_excess_four_or_more` DataFrame to an Excel file named 'enhanced_contract_demand.xlsx' and then use `files.download` to provide a download link for the user.

In [10]:
from google.colab import files

# Save the DataFrame to an Excel file
output_filename = 'enhanced_contract_demand.xlsx'
df_excess_four_or_more.to_excel(output_filename, index=False)

# Provide a download link for the user
files.download(output_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>